In [2]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# STEP 2
## IFC Data Collection

In [3]:
# ifc_file = ifcopenshell.open('1-PlatformBaseQty.ifc') # Manual input
ifc_file = ifcopenshell.open('Proof_of_concept.ifc') # Manual input
custom_property_name = 'Preassembly' # Manual input

### Units of measure and conversion factor

In [4]:
project = ifc_file.by_type('IfcProject')[0] # Get the IfcProject entity
unit_assignment = project.UnitsInContext # Get the IfcUnitAssignment entity

# Get the length unit
length_unit = None
for unit in unit_assignment.Units: 
# Units retrieves IfcSIUnit, IfcConversionBasedUnit and IfcDerivedUnit
    if unit.is_a('IfcSIUnit') and unit.UnitType == 'LENGTHUNIT':
        length_unit = unit
        break

# Get the mass unit
mass_unit = None
for unit in unit_assignment.Units:
    if unit.is_a('IfcSIUnit') and unit.UnitType == 'MASSUNIT':
        mass_unit = unit
        break
# Get the volume unit
volume_unit = None
for unit in unit_assignment.Units:
    if unit.is_a('IfcSIUnit') and unit.UnitType == 'VOLUMEUNIT':
        volume_unit = unit
        break
# Get the mass density unit
massdensity_unit = None
for unit in unit_assignment.Units:
    if unit.is_a('IfcDerivedUnit') and unit.UnitType == 'MASSDENSITYUNIT':
        massdensity_unit = unit
        break

In [5]:
def lengthunit_conversionfactor(unit):
    if unit:
        if unit.Prefix == 'MILLI':
            lengthunit_conversion_factor = 0.001
        elif unit.Prefix == 'CENTI':
            lengthunit_conversion_factor = 0.01
        elif unit.Prefix == 'DECI':
            lengthunit_conversion_factor = 0.1
        elif unit.Prefix == 'DECA':
            lengthunit_conversion_factor = 10
        elif unit.Prefix == 'HECTO':
            lengthunit_conversion_factor = 100
        elif unit.Prefix == 'KILO':
            lengthunit_conversion_factor = 1000
        else:
            lengthunit_conversion_factor = 1 # METER
    else:
        print('No length unit found in the IFC file')
        
    return lengthunit_conversion_factor

length_cf = lengthunit_conversionfactor(length_unit)

In [6]:
def massunit_conversionfactor(unit):
    if unit:
        if unit.Prefix == 'MILLI':
            massunit_conversion_factor = 0.000001
        elif unit.Prefix == 'CENTI':
            massunit_conversion_factor = 0.00001
        elif unit.Prefix == 'DECI':
            massunit_conversion_factor = 0.0001
        elif unit.Prefix == 'DECA':
            massunit_conversion_factor = 0.01
        elif unit.Prefix == 'HECTO':
            massunit_conversion_factor = 0.1
        elif unit.Prefix == 'KILO':
            massunit_conversion_factor = 1
        else:
            massunit_conversion_factor = 0.001 # GRAM
    else:
        print('No mass unit found in the IFC file')
    
    return massunit_conversion_factor

mass_cf = massunit_conversionfactor(mass_unit)

In [7]:
massdensity_cf = 1
massdensitystring = ''
if massdensity_unit:
    for element in massdensity_unit.Elements:
    # Elements = subunits of the IfcDerived
        if element.Unit.UnitType == 'MASSUNIT':
            massdensity_cf *= massunit_conversionfactor(element.Unit) ** element.Exponent
            massdensitystring += '(KILOGRAM^' + str(element.Exponent) + ')'
        elif element.Unit.UnitType == 'LENGTHUNIT':
            massdensity_cf *= lengthunit_conversionfactor(element.Unit) ** element.Exponent
            massdensitystring += '(METER^' + str(element.Exponent) + ')'
else:
    print('No mass density unit found in the IFC file')

### Preassembly identification

In [8]:
preassembly_values = set()
for entity in ifc_file.by_type('IfcProduct'):
    for definition in entity.IsDefinedBy:
        if definition.is_a('IfcRelDefinesByProperties'):
            property_definition = definition.RelatingPropertyDefinition 
            if property_definition.is_a('IfcPropertySet'):
            # It could be IfcPropertySet or IfcElementQuantity
                for property in property_definition.HasProperties:
                    if property.Name == custom_property_name:
                        preassembly_values.add(property.NominalValue.wrappedValue)
            elif property_definition.is_a('IfcElementQuantity'):
                pass
# Create DataFrames
column_names = list(preassembly_values) # eliminates repeated values
row_names = ['Dim X [m]', 'Dim Y [m]', 'Dim Z [m]', 'CG_X [m]', 'CG_Y [m]', 'CG_Z [m]', 'Load Length [m]', 'Load Width [m]', 'Load Height [m]', 'Load Mass [kg]']
data = [[None] * len(column_names)] * len(row_names)
DataFrame = pd.DataFrame(data, columns=column_names, index=row_names)
GlobalIds = pd.DataFrame(columns=column_names)

In [9]:
# Find GUID of elements per steel preassembly
for value in preassembly_values:
    globalids = []
    for entity in ifc_file.by_type('IfcProduct'):
        for definition in entity.IsDefinedBy:
            if definition.is_a('IfcRelDefinesByProperties'):
                property_definition = definition.RelatingPropertyDefinition
                if property_definition.is_a('IfcPropertySet'):
                    for property in property_definition.HasProperties:
                        if property.Name == custom_property_name and property.NominalValue.wrappedValue == value:
                            globalids.append(entity.GlobalId)
                elif property_definition.is_a('IfcElementQuantity'):
                    pass

    # Place each GlobalId in the corresponding column
    for i, globalid in enumerate(globalids):
        GlobalIds.at['GlobalID_' + str(i+1), value] = globalid

# print(DataFrame)
# print(GlobalIds)

### Mass density and gross volume

In [10]:
def get_mass_density(ifc_file,massdensity_cf,globalid):

    #Retrieve the entity that has the global id
    product = ifc_file.by_guid(globalid)
   
    #Initiate density variable
    density = 0
    # Search for product's material
    for association in product.HasAssociations:
        if association.is_a('IfcRelAssociatesMaterial'):
            product_material = association.RelatingMaterial
            if product_material.is_a('IfcMaterial'):
                # Search for entity that links product's material with its properties
                for related_material in ifc_file.by_type('IfcRelAssociatesMaterial'):
                    if related_material.RelatingMaterial == product_material:
                        # Search for the entity with 'Density' property
                        for ext_mat_property in ifc_file.by_type('IfcExtendedMaterialProperties'): 
                            if ext_mat_property.Material == product_material:
                                for property in ext_mat_property.ExtendedProperties:
                                    if property.Name == 'Density':
                                        density = property.NominalValue.wrappedValue*massdensity_cf
    
    return density 

In [11]:
def get_gross_volume(ifc_file,globalid):
     
    product = ifc_file.by_guid(globalid)
    if product.IsDefinedBy:
        definitions = product.IsDefinedBy
        # Either IfcRelDefinesByProperties or IfcRelDefinesByType        
        for definition in definitions:            
            if 'IfcRelDefinesByProperties' == definition.is_a():
                property_definition = definition.RelatingPropertyDefinition
            # Either IfcPropertySet or IfcElementQuantity
                if 'IfcElementQuantity' == property_definition.is_a():
                    for quantity in property_definition.Quantities:
                        if 'IfcQuantityVolume' == quantity.is_a() and quantity.Name == 'GrossVolume':
                            volume_value = quantity.VolumeValue
                                                
            if 'IfcRelDefinesByType' == definition.is_a():
                type = definition.RelatingType
                if type.HasPropertySets:
                    for property_definition in type.HasPropertySets:
                        if 'IfcElementQuantity' == property_definition.is_a():
                            for quantity in property_definition.Quantities:
                                if 'IfcQuantityVolume' == quantity.is_a() and quantity.Name == 'GrossVolume':
                                    volume_value = quantity.VolumeValue
    return volume_value

### Geometry and spatial location

In [12]:
def get_vertices_localcoordinates(ifc_file,globalid):
    
    settings = ifcopenshell.geom.settings()
    element = ifc_file.by_guid(globalid)
    
    if element is not None:
        try:
            # Geometric representation of the element, always in METER
            shape = ifcopenshell.geom.create_shape(settings, element)
            # (xl,yl,zl) of vertices in list, always in LOCAL SYSTEM
            vertices = np.array(shape.geometry.verts).reshape((-1, 3))
        except RuntimeError:                                            
            print(f"No geometric representation for entity with GlobalId {globalid}")
    else:
        print(f"No element found with the provided GlobalId {globalid}")
    
    return vertices

In [13]:
def get_vertices_globalcoordinates(ifc_file,length_cf,globalid):

    settings = ifcopenshell.geom.settings()
    element = ifc_file.by_guid(globalid)
    if element is not None:
        try:
            # Geometric representation of the element, always in METER
            shape = ifcopenshell.geom.create_shape(settings, element)
            # (xl,yl,zl) of vertices in list, always in LOCAL SYSTEM
            vertices = np.array(shape.geometry.verts).reshape((-1, 3))
            # Matrix to go from LOCAL to GLOBAL coordinates, in LENGTHUNIT
            matrix = ifcopenshell.util.placement.get_local_placement(element.ObjectPlacement) #4x4 matrix
            # Transform location of the element (4th column) to METER
            matrix[:3, 3] *= length_cf
            # Apply transformation matrix --> GLOBAL coordinates                
            vertices = [np.dot(matrix, np.append(vertex, 1))[:3] for vertex in vertices]                                      
        except RuntimeError:
            print(f"No geometric representation for entity with GlobalId {globalid}")
    else:
        print(f"No element found with the provided GlobalId {globalid}")
    
    return vertices

# STEP 3
## Derivation of implicit information
### Bounding box

In [14]:
def get_bbox(vertices):
    
    # Find the minimum and maximum coordinates
    min_coord = vertices.min(axis=0)
    max_coord = vertices.max(axis=0)

    # The min and max coordinates represent the bounding box
    dim_x = max_coord[0] - min_coord[0]
    dim_y = max_coord[1] - min_coord[1]
    dim_z = max_coord[2] - min_coord[2]
    
    return dim_x, dim_y, dim_z


In [15]:
# Function to plot each set of vertices
def plot_vertices(vertices,column):
    
    # Extract the X, Y and Z coordinates
    x_coords = vertices[:, 0]
    y_coords = vertices[:, 1]
    z_coords = vertices[:, 2]
    
    # Create a 3D scatter plot of the vertices
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_coords, y_coords, z_coords)
    ax.set_title(column)
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    ax.set_box_aspect([np.ptp(x_coords), np.ptp(y_coords), np.ptp(z_coords)])  # Make axes equal scale 
    plt.subplots_adjust(left=0.1, right=0.8, top=0.9, bottom=0.1) # Adjust the position of the axes within the figure
    plt.show()


In [27]:
for column in DataFrame.columns:
    
    globalids = []
    all_vertices = []
    
    # Create list of global ids of the preassembly
    for index, value in GlobalIds[column].items():
        if index.startswith('GlobalID_') and pd.notna(value):
            globalids.append(value)
    
    # Retrieve all the global coordinates of the preassembly's vertices        
    for globalid in globalids:
        vertices = get_vertices_globalcoordinates(ifc_file,length_cf,globalid)
        all_vertices.extend(vertices)
        
    # Convert list of all vertices to a numpy array for easier manipulation
    all_vertices = np.array(all_vertices) 

    # Obtain the bounding box = three dimensions in x-, y- and z- direction
    dim_x, dim_y, dim_z = get_bbox(all_vertices)

    # Place each coordinate in the corresponding cell of the Data Frame
    DataFrame.at['Dim X [m]', column] = dim_x
    DataFrame.at['Dim Y [m]', column] = dim_y
    DataFrame.at['Dim Z [m]', column] = dim_z
    
    #plot_vertices(all_vertices,column)

#print(DataFrame)
#print(GlobalIds)

### Centre of gravity

In [17]:
def get_mass(ifc_file,massdensity_cf,globalid):
     
    volume_value = get_gross_volume(ifc_file,globalid)
    mass_density_value = get_mass_density(ifc_file,massdensity_cf,globalid)
    mass_value = volume_value * mass_density_value
    
    return mass_value

In [18]:
def get_centreofmass_globalcoordinates(ifc_file,length_cf,globalid):

    vertices = get_vertices_globalcoordinates(ifc_file,length_cf,globalid)

    centroid_x = sum(vertex[0] for vertex in vertices) / len(vertices)
    centroid_y = sum(vertex[1] for vertex in vertices) / len(vertices)
    centroid_z = sum(vertex[2] for vertex in vertices) / len(vertices)
    
    return centroid_x, centroid_y, centroid_z

In [19]:
def get_centre_gravity(ifc_file,massdensity_cf,length_cf,globalids):
    total_mass = sum(get_mass(ifc_file,massdensity_cf,globalid) for globalid in globalids)

    x_coords = []
    y_coords = []
    z_coords = []
    all_vertices = []
    
    for globalid in globalids:
        # Numerator of CG formula (r*mass)
        mass = get_mass(ifc_file,massdensity_cf,globalid)
        centroid = get_centreofmass_globalcoordinates(ifc_file,length_cf,globalid)
        x_coords.append(centroid[0] * mass)
        y_coords.append(centroid[1] * mass)
        z_coords.append(centroid[2] * mass)
        
        # Minimum coordinates of vertices 
        vertices = get_vertices_globalcoordinates(ifc_file,length_cf,globalid)
        all_vertices.extend(vertices)      

    all_vertices = np.array(all_vertices)
    min_coord = all_vertices.min(axis=0)

    cg_x = sum(x_coords) / total_mass - min_coord[0]
    cg_y = sum(y_coords) / total_mass - min_coord[1]
    cg_z = sum(z_coords) / total_mass - min_coord[2]
  
    return cg_x, cg_y, cg_z, total_mass

In [20]:
for column in DataFrame.columns:

    # Obtain list of global ids of the preassembly
    globalids = [GlobalIds.at[index, column] for index in GlobalIds.index if index.startswith('GlobalID_') and pd.notna(GlobalIds.at[index, column])]

    # Obtain centre of gravity and mass of the preassembly
    cg_x, cg_y, cg_z, total_mass = get_centre_gravity(ifc_file,massdensity_cf,length_cf,globalids)

    # Place each value in the corresponding cell of the DataFrame
    DataFrame.at['CG_X [m]', column] = cg_x
    DataFrame.at['CG_Y [m]', column] = cg_y
    DataFrame.at['CG_Z [m]', column] = cg_z
    DataFrame.at['Load Mass [kg]', column] = total_mass

# print(DataFrame)

## Characteristics of a loaded vehicle
### Transport position of a preassembly (the load)

In [21]:
for column in DataFrame.columns:    
    # Bounding box
    dim_x = DataFrame.at['Dim X [m]', column]
    dim_y = DataFrame.at['Dim Y [m]', column]
    dim_z = DataFrame.at['Dim Z [m]', column]
    # Centre of gravity
    cg_x = DataFrame.at['CG_X [m]', column]
    cg_y = DataFrame.at['CG_Y [m]', column]
    cg_z = DataFrame.at['CG_Z [m]', column]
    # Long sections long the length
    length = max(dim_x,dim_y,dim_z)
    if length == dim_x:
        l1 = min(cg_x,length-cg_x)/dim_x # CG relative in length
        height = min(dim_y,dim_z)
        width = max(dim_y,dim_z)
        if height == dim_z: # width = dim_y
            cg_w = min(cg_y,width-cg_y)
            cg_h = min(cg_z,height-cg_z)
            w1 = cg_w/dim_y # max 50%
            h1 = cg_h/dim_z # max 50%    
        else: # height = dim_y, width = dim_z
            cg_w = min(cg_z,width-cg_z)
            cg_h = min(cg_y,height-cg_y)
            h1 = cg_h/dim_y # max 50%
            w1 = cg_w/dim_z # max 50%
    
    elif length == dim_y:
        l1 = min(cg_y,length-cg_y)/dim_y
        height = min(dim_x,dim_z)
        width = max(dim_x,dim_z)
        if height == dim_z: # width = dim_x
            cg_w = min(cg_x,width-cg_x)
            cg_h = min(cg_z,height-cg_z)
            w1 = cg_w/dim_x # max 50%
            h1 = cg_h/dim_z # max 50%    
        else: # height = dim_x, width = dim_z
            cg_w = min(cg_z,width-cg_z)
            cg_h = min(cg_x,height-cg_x)
            h1 = cg_h/dim_x # max 50%
            w1 = cg_w/dim_z # max 50%
    
    else: # length = dim_z
        l1 = min(cg_z,length-cg_z)/dim_z
        height = min(dim_x,dim_y)
        width = max(dim_x,dim_y)
        if height == dim_x: # width = dim_y
            cg_w = min(cg_y,width-cg_y)
            cg_h = min(cg_x,height-cg_x)
            h1 = cg_h/dim_x # max 50%
            w1 = cg_w/dim_y # max 50%    
        else: # height = dim_y, width = dim_x
            cg_w = min(cg_x,width-cg_x)
            cg_h = min(cg_y,height-cg_y)
            w1 = cg_w/dim_x # max 50%
            h1 = cg_h/dim_y # max 50%
    
    # Centre of gravity as low as possible
    # And centered in transverse direction
    
    if h1 <= w1 and cg_h <= cg_w: # Do not swap
        if round(w1,1) < 0.5:
            print('Alert: CG not centered in transverse direction (',w1*100,'%)')
    
    if w1 < h1 and cg_w < cg_h: # Swap
        height, width = width, height
        h1, w1, cg_w, cg_h = w1, h1, cg_h, cg_w
        if round(w1,1) < 0.5:
            print('Alert: CG not centered in transverse direction (',w1*100,'%)')
    
    # Store the calculated dimensions and new CG into the DataFrame
    DataFrame.at['Load Length [m]', column] = length
    DataFrame.at['Load Width [m]', column] = width
    DataFrame.at['Load Height [m]', column] = height
    DataFrame.at['Load CG_length [-]', column] = l1
    DataFrame.at['Load CG_width [-]', column] = w1
    DataFrame.at['Load CG_height [-]', column] = h1
    
print(DataFrame) # Importan to print in this step, to know which vehicle to use.

                           TR01         TW01        PF01
Dim X [m]                  8.96         6.48         2.8
Dim Y [m]                 1.743         3.33        3.33
Dim Z [m]                  0.25         15.0         3.0
CG_X [m]               4.478119     3.858931    1.389984
CG_Y [m]                0.67112     1.656687    1.676874
CG_Z [m]               0.133705     8.186539    2.170868
Load Length [m]            8.96         15.0        3.33
Load Width [m]            1.743         6.48         2.8
Load Height [m]            0.25         3.33         3.0
Load Mass [kg]      1735.067411  15930.45418  2445.47311
Load CG_length [-]      0.49979     0.454231    0.496434
Load CG_width [-]      0.385037     0.404486    0.496423
Load CG_height [-]     0.465181     0.497504    0.276377


### Loaded vehicle

In [22]:
for column in DataFrame.columns:
    # Vehicle information - manual input
    while True:
        Vehicle = input(f"Enter Vehicle for {column} (single/tow): ")
        if Vehicle.lower() in ['single', 'tow']:
            break
        else:
            print("Invalid input. Please enter 'single' or 'tow'.")
    if Vehicle == 'single':
            Vehicle_type = 'truck'
    else:
        while True:
            Vehicle_type = input(f"Enter Vehicle type for {column} (tractor+semitrailer/truck+trailer/LHV): ")
            if Vehicle_type.lower() in ['tractor+semitrailer', 'truck+trailer', 'LHV']:
                break
            else:
                print("Invalid input. Please enter 'tractor+semitrailer', 'truck+trailer' or 'LHV.")
    while True:
        try:
            Lveh = float(input(f"Enter length of vehicle for {column} in [m]: "))
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            Lveh_bed = float(input(f"Enter length of vehicle's bed for {column} in [m]: "))
            Lveh_front = Lveh - Lveh_bed
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            Wveh = float(input(f"Enter width of vehicle for {column} in [m]: ")) 
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            Hveh = float(input(f"Enter height of vehicle for {column} in [m]: "))
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            Hveh_bed = float(input(f"Enter height of vehicle's bed for {column} in [m]: "))
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            LoadSupports = float(input(f"Enter height of supports for {column} in [m]: "))
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")
    while True:
        try:
            Mveh = float(input(f"Enter mass of vehicle for {column} in [kg] (include load supports): "))
            break
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    # Store manual input into the DataFrame
    DataFrame.at['Vehicle', column] = Vehicle
    DataFrame.at['Vehicle type', column] = Vehicle_type
    DataFrame.at['Lveh [m]', column] = Lveh
    DataFrame.at['Lveh_bed [m]', column] = Lveh_bed
    DataFrame.at['Lveh_front [m]', column] = Lveh_front
    DataFrame.at['Wveh [m]', column] = Wveh
    DataFrame.at['Hveh [m]', column] = Hveh 
    DataFrame.at['Hveh_bed [m]', column] = Hveh_bed
    DataFrame.at['Mveh [kg]', column] = Mveh
    DataFrame.at['LoadSupports [m]', column] = LoadSupports
    
    # Load information - from Data Frame
    Lload = DataFrame.at['Load Length [m]', column]    
    Wload = DataFrame.at['Load Width [m]', column]
    Hload = DataFrame.at['Load Height [m]', column]
    Mload = DataFrame.at['Load Mass [kg]', column]
    
    # Loaded vehicle mass and dimensions
    L = Lveh_front + max(Lveh_bed,Lload)
    W = max(Wveh,Wload)
    H = max(Hveh,Hveh_bed + LoadSupports + Hload)
    RearOverhang = L - Lveh
    M = Mveh + Mload
    
    # Store final mass and dimensions of loaded vehicle into the DataFrame
    DataFrame.at['Length [m]', column] = L
    DataFrame.at['Width [m]', column] = W
    DataFrame.at['Height [m]', column] = H
    DataFrame.at['Mass [kg]', column] = M            
    DataFrame.at['RearOverhang [m]', column] = RearOverhang
    
print(DataFrame)

Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
                                   TR01                 TW01         PF01
Dim X [m]                          8.96                 6.48          2.8
Dim Y [m]                         1.743                 3.33         3.33
Dim Z [m]                          0.25                 15.0          3.0
CG_X [m]                       4.478119             3.858931     1.389984


## Rule encoding

In [23]:
# Computer language encoded rules 
# Interpreted and translated normative language

# Admissible width
def dec_table_1(Wveh):
    if Wveh == 2.60:
        Wadm = 2.6
    else:
        Wadm = 2.55
    return Wadm


# RearOverhang
def dec_table_2(L, Ladm, RearOverhang):
    if L <= Ladm and RearOverhang <= 1:
        return "OK"
    elif L <= Ladm and RearOverhang > 1:
        return "ALERT: exceeded"
    elif L > Ladm and RearOverhang <= 3:
        return "OK"
    else:
        return "ALERT: exceeded"


# Transport type
def dec_table_3(L, W, H, M, Ladm, Wadm, Hadm, Madm):
    if L <= Ladm and W <= Wadm and H <= Hadm and M <= Madm:
        transport = "normal"
    else:
        transport = "exceptional"
    return transport

    
# Dimensions exceeded   
def dec_table_4(L, W, H, M, Ladm, Wadm, Hadm, Madm):
    DimExceeded = 0
    messages = []
    if L > Ladm:
        DimExceeded += 1
        messages.append("Length")
    if W > Wadm:
        DimExceeded += 1
        messages.append("Width")
    if H > Hadm:
        DimExceeded += 1
        messages.append("Height")
    if M > Madm:
        messages.append("Mass")
    if DimExceeded == 0:
        messages.append("None")
    return DimExceeded, messages

    
# Admisible length
def dec_table_5(Vehicle_type):
    if Vehicle_type == "truck":
        Ladm = 12
        L1, L2, L3 = 19, 22, 28
    elif Vehicle_type == "tractor+semitrailer":
        Ladm = 16.5
        L1, L2, L3 = 27, 30, 35
    elif Vehicle_type == "truck+trailer":
        Ladm = 18.75
        L1, L2, L3 = 27, 30, 35
    elif Vehicle_type == "LHV":
        Ladm = 25
        L1, L2, L3 = 27, 30, 35
    else:
        Ladm = L1 = L2 = L3 = None
    return Ladm, L1, L2, L3


# Exceptional vehicle category
def dec_table_6(L, W, H, M, Ladm, Wadm, Hadm, Madm, L1, L2, L3):
    category = None
    if L > L3 or W > 5 or H > 4.8 or M > 120000:
        category = 4
    elif L2 < L <= L3 or 4.25 < W <= 5 or 4.5 < H <= 4.8 or 90000 < M <= 120000:
        category = 3
    elif L1 < L <= L2 or 3.5 < W <= 4.25 or Hadm < H <= 4.5 or Madm < M <= 90000:
        category = 2
    elif Ladm < L <= L1 and Wadm < W <= 3.5:
        category = 1
    else:
        category = "N/A"
    return category


# Type of permit and validity
def dec_table_7(category):
    permit = []
    validity = []
    if category == 1:
        permit = "Long-term, permanent"
        validity = "5 years"
    elif category == 2:
        permit = "Long-term, permanent"
        validity = "1 year"
    elif category == 3:
        permit = "Short-term, temporary"
        validity = "4 months"
    elif category == 4:
        permit = "Short-term, temporary"
        validity = "2 months"
    else:
        permit = "N/A"
        validity = "N/A"
    return permit, validity

# Steered axles
def dec_table_8(Vehicle, L):
    if Vehicle == "single" and L > 19:
        return "ALERT: At least one at the front and rear of the single vehicle"
    elif Vehicle == "tow" and L > 27:
        return "ALERT: At least one on the longest towed vehicle"
    else:
        return "OK, no requirement"


# Multiple indivisible loads
def dec_table_9(transport, L, W, H, M, Ladm, Wadm, Hadm, Madm):
    if transport == "exceptional":
        MultiplePartsLength = "NO"
        MultiplePartsWidth = "NO"
        MultiplePartsHeight = "NO"
        message = "N/A"
        if L <= Ladm and M <= Madm: # W or H exceeded
            MultiplePartsLength = "YES"
        if W <= Wadm and M <= Madm: # H or L exceeded
            MultiplePartsWidth = "YES"
        if H <= Hadm and M <= Madm:  # W or L exceeded
            MultiplePartsHeight = "YES"
        if L > Ladm:
            message = "NOTE: Attach technical note to the permit application with technical or stability reasons if transported simultaneously with other long elements."
    else:
        MultiplePartsLength = "N/A"
        MultiplePartsWidth = "N/A"
        MultiplePartsHeight = "N/A"
        message = "N/A"
    return MultiplePartsLength, MultiplePartsWidth, MultiplePartsHeight, message


# Number of exceptional dimensions
def dec_table_10(transport, DimExceeded, W, H, Wadm, Hadm):
    if transport == "exceptional":
        if DimExceeded == 0:
            message1 = "N/A"
            message2 = "N/A"
        elif DimExceeded > 0:
            if W > Wadm:
                message1 = "To reduce the exceptional width, one element of the indivisible load can be dismantled and placed on the same vehicle, even if it increases the original length (exceptional or not)."
                message2 = "To reduce the exceptional width, the load can be tilted, even if it increases the original height (exceptional or not)."
            elif H > Hadm:
                message1 = "To reduce the exceptional height, one element of the indivisible load can be dismantled and placed on the same vehicle, even if it increases the original length (exceptional or not)."
                message2 = "To reduce the exceptional height, the load can be tilted, even if it increases the original width (exceptional or not)."
    else:
        message1 = "N/A"
        message2 = "N/A"
    return message1, message2


# Alternative loading method  
def dec_table_11(transport, MultiplePartsLength, Lveh, Wveh, Hveh, Mveh, M, Ladm, Wadm, Hadm, Madm):
    if transport == "exceptional" and MultiplePartsLength == "YES":
        if Lveh <= Ladm and Wveh <= Wadm and Hveh <= Hadm and Mveh <= Madm and M <= Madm:
            message = "The carrier may, for efficiency reasons, position the indivisible loads in such a way that there is one additional exceptional dimension, in height up to 4.3m or width up to 3m, as long as: (1) by placing the load in this way, it can be demonstrated that at least 30% more can be transported than if the height or width remained within the permitted values; (2) the alternative loading method does not pose any additional risk to road safety."
        else:
            message = "N/A"
    else:
        message = "N/A"
    return message


# Guidance
def dec_table_12(transport, L, W, H, M):
    if transport == "exceptional":
        if L > 40 or W > 5:
            return "3 escort vehicles + 1 traffic coordinator"
        elif 35 < L <= 40 or 4.5 < W <= 5 or H > 4.8 or M > 180000:
            return "2 escort vehicles + 1 traffic coordinator"
        elif 30 < L <= 35 or 3.5 < W <= 4.5 or 90000 < M <= 180000:
            return "1 escort vehicle + 1 traffic coordinator"
        else:
            return "N/A"
    else:
        return "N/A"

## Rule execution

In [24]:
for column in DataFrame.columns:
    
    # Arguments - from data frame
    L = DataFrame.at['Length [m]', column]
    W = DataFrame.at['Width [m]', column]
    H = DataFrame.at['Height [m]', column]
    M = DataFrame.at['Mass [kg]', column]
    Lveh = DataFrame.at['Lveh [m]', column]
    Wveh = DataFrame.at['Wveh [m]', column]
    Hveh = DataFrame.at['Hveh [m]', column]
    Mveh = DataFrame.at['Mveh [kg]', column]
    Vehicle = DataFrame.at['Vehicle', column]
    Vehicle_type = DataFrame.at['Vehicle type', column]
    RearOverhang = DataFrame.at['RearOverhang [m]', column]
    
    # Arguments/values - from functions
    Ladm, L1, L2, L3 = dec_table_5(Vehicle_type) # values
    Wadm = dec_table_1(Wveh) # value
    Hadm = 4
    Madm = 44000
    CheckRearOverhang = dec_table_2(L, Ladm, RearOverhang) # message
    transport = dec_table_3(L, W, H, M, Ladm, Wadm, Hadm, Madm) # message
    DimExceeded_v, DimExceeded_m = dec_table_4(L, W, H, M, Ladm, Wadm, Hadm, Madm) # value, message
    category = dec_table_6(L, W, H, M, Ladm, Wadm, Hadm, Madm, L1, L2, L3)
    permit, validity = dec_table_7(category) # messages
    steered_axles = dec_table_8(Vehicle, L) # message
    MultiplePartsLength, MultiplePartsWidth, MultiplePartsHeight, multiple_m = dec_table_9(transport, L, W, H, M, Ladm, Wadm, Hadm, Madm) # messages
    dismantle, tilt = dec_table_10(transport, DimExceeded_v, W, H, Wadm, Hadm) # messages
    alt_method = dec_table_11(transport, MultiplePartsLength, Lveh, Wveh, Hveh, Mveh, M, Ladm, Wadm, Hadm, Madm) # message
    guidance = dec_table_12(transport, L, W, H, M) # message
    
    # Store into the DataFrame
    DataFrame.at['Ladm [m]', column] = Ladm
    DataFrame.at['Wadm [m]', column] = Wadm
    DataFrame.at['Hadm [m]', column] = Hadm
    DataFrame.at['Madm [kg]', column] = Madm
    DataFrame.at['Transport (normal/exceptional)', column] = transport
    DataFrame.at['RearOverhang check', column] = CheckRearOverhang
    DataFrame.at['Dimensions exceeded', column] = DimExceeded_m
    DataFrame.at['Exceptional transport category', column] = category
    DataFrame.at['Permit required', column] = permit
    DataFrame.at['Permit validity', column] = validity
    DataFrame.at['Steered axles requirement', column] = steered_axles
    DataFrame.at['Multiple parts allowed in length', column] = MultiplePartsLength
    DataFrame.at['Multiple parts allowed in width', column] = MultiplePartsWidth
    DataFrame.at['Multiple parts allowed in height', column] = MultiplePartsHeight
    DataFrame.at['Multiple parts allowed - additional', column] = multiple_m
    DataFrame.at['Dimensions exceeded - note 1', column] = dismantle
    DataFrame.at['Dimensions exceeded - note 2', column] = tilt
    DataFrame.at['Alternative loading method', column] = alt_method
    DataFrame.at['Guidance required', column] = guidance
    
print(DataFrame)

                                                    TR01  \
Dim X [m]                                           8.96   
Dim Y [m]                                          1.743   
Dim Z [m]                                           0.25   
CG_X [m]                                        4.478119   
CG_Y [m]                                         0.67112   
CG_Z [m]                                        0.133705   
Load Length [m]                                     8.96   
Load Width [m]                                     1.743   
Load Height [m]                                     0.25   
Load Mass [kg]                               1735.067411   
Load CG_length [-]                               0.49979   
Load CG_width [-]                               0.385037   
Load CG_height [-]                              0.465181   
Vehicle                                              tow   
Vehicle type                         tractor+semitrailer   
Lveh [m]                                

# STEP 4: Reporting of compliance checking

In [25]:
import os
from openpyxl import Workbook
from openpyxl.styles import Alignment, Border, Side

# Define name of output file   
project_name = project.LongName
output_file_name = f"{project_name}_{custom_property_name}_ComplianceChecking.xlsx"

# Create an Excel writer object using pandas
try:
    writer = pd.ExcelWriter(output_file_name, engine='openpyxl')
except PermissionError:
    print("File already open, please close it to run the code block.")
# Write the data frames to the Excel writer
DataFrame.to_excel(writer, sheet_name='ComplianceChecking')
GlobalIds.to_excel(writer, sheet_name='GlobalIds')
# Define style
workbook = writer.book
thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))
# Define the alignment styles
middle_align_text = Alignment(vertical='center', wrap_text=True)
left_align_text = Alignment(horizontal='left', vertical='center', wrap_text=True)
center_align_text = Alignment(horizontal='center', vertical='center', wrap_text=True)
# Function to apply styles to sheets
def style_sheet(sheet):
    # Apply the styles to each cell
    for row in sheet.iter_rows(min_row=1, max_col=sheet.max_column, max_row=sheet.max_row):
        for cell in row:
            cell.border = thin_border
            cell.alignment = middle_align_text
    # Apply left alignment to the first column and center alignment to the rest
    for cell in sheet['A']:
        cell.alignment = left_align_text
    for col in sheet.iter_cols(min_col=2, max_col=sheet.max_column, min_row=1, max_row=sheet.max_row):
        for cell in col:
            cell.alignment = center_align_text
    # Set column width
    for col in sheet.columns:
        column = col[0].column_letter  # Get the column name
        sheet.column_dimensions[column].width = 43.67
# Apply styles to sheets
style_sheet(writer.sheets['ComplianceChecking'])
style_sheet(writer.sheets['GlobalIds'])

# Close the workbook to save it
try:
    writer.close()
    print("Compliance checking ready, please find the report in the file",output_file_name, "located at",os.getcwd())
except:
    print("File already open, please close it to run the code block.")

Compliance checking ready, please find the report in the file ProofOfConcept_Preassembly_ComplianceChecking.xlsx located at c:\Users\const\UGENT\0_THESIS\8_Codes
